# Exercise Set 2
**Q1.** Re-write the **Charge/discharge:** constraints using indicator constraints. Remember to consider what happens when $z_{b,t} = 1$ and when $z_{b,t} = 0$

Define $z_{b,t} \in \{0,1\}, t \in T$.
$$
\begin{align*}
f^{in}_{b,t} &\leq 20*z_{b,t} &\forall b \in B, t \in T \\
f^{out}_{b,t} &\leq 20*(1-z_{b,t}) &\forall b \in B, t \in T
\end{align*}
$$

In [ ]:
m.addConstrs((((z[b,t] == 0) >> (flow_in[b,t] == 0)) for b in batteries for t in time_periods), name = "zis0")
m.addConstrs((((z[b,t] == 1) >> (flow_out[b,t] == 0)) for b in batteries for t in time_periods), name = "zis1")



The multiple scenarios API is restricted. For example, it is not possible to explicitly
- Add/remove variables or constraints
- Change the variable types
- Change the sense of constraints

However, we can get around some of the restrictions using some tricks.

Assume `ScenarioNumber` and `ScenNName` parameters are already set for each of the below.
______
To remove a variable, set its bounds to zero.

**Q2.** Write a `for` loop and `gurobipy` code to remove the flow in and flow out for `Battery0` in time periods 0 to 49.
______
To add a variable to a scenario, add it to the base model with bounds set to zero and then change the bounds accordingly.

**Q3.** Write `gurobipy` code to add a continuous variable $w_t, t\in T$ to the base model `m` such that it's "not in" this model as described above. Then write a `for` loop to add it to a scenario with an *upper bound* of 100.
______

To remove a constraint, change its RHS values to GRB.INFINITY/-GRB.INFINITY.

**Q4.** We defined $v_{b,t} = 1$ when battery $b$ is below a depth of discharge α (say 0.3) and 0 otherwise. Supposed we had a hard constraint on the number of times both batteries together could go below their depth, with that limit to be 15. And say we stored (set the constraint equal to) this constraint as `depth_limit`.

$$
\begin{equation}
\sum_{b,t} v_{b,t} \le C
\end{equation}
$$

Add a constraint to the base model `m` for the number of times either battery could go below its depth. Write the `gurobipy` code to remove this constraint.
______

In [ ]:
#Q2.
for t in range(50):
  flow_in['Battery0',t].ScenNUB = 0
  flow_out['Battery0',t].ScenNUB = 0

#Q3.
w = m.addVars(time_periods, lb = 0, ub = 0, name = 'w')
#....
for t in time_periods:
  w[t].ScenNUB = 100

#Q4.
v = m.addVars(time_periods, vtype=GRB.BINARY, name = 'v')
# define a linear expression for total depth count
total_depth_count = v.sum() # this is optional!
depth_limit = m.addConstr(total_depth_count <= 15, name = 'depth_limit')

# remove the constraint by setting the RHS to infinity
depth_limit.ScenNRhs = GRB.INFINITY